In [41]:
import gzip
import numpy as np
import _pickle as cPickle

In [42]:
totalClasses = 10
LR = 0.1
LS = 0.5
epochs = 5
EF = 10000

In [43]:
dataTrain, dataValidate, dataTest = cPickle.load(gzip.open('mnist.pkl.gz', 'rb'),encoding='latin1')

In [44]:
xTrain = dataTrain[0].T
yTrain = dataTrain[1]

In [45]:
xValidate = dataValidate[0].T
yValidate = dataValidate[1]

In [46]:
xTest = dataTest[0].T
yTest = dataTest[1]

In [47]:
(Ktrain,    Ntrain)    = xTrain.shape
(Kvalidate, Nvalidate) = xValidate.shape
(Ktest,     Ntest)     = xTest.shape

In [48]:
###### Fully connected layer 1
K1 = Ktrain
M1 = 100

#bias
K1b = M1
M1b = K1b

#relu
K1r = M1b
M1r = K1r 


##### Fully connected layer 2
K2 = M1r
M2 = totalClasses

#bias
K2b = M2
M2b = K2b

In [49]:
w0 = np.random.randn(M1, K1)
w1 = np.random.randn(K1b)
w3 = np.random.randn(M2, K2)
w4 = np.random.randn(K2b)

In [50]:
x0 = np.zeros(K1)
x1 = np.zeros(K1b)
x2 = np.zeros(K1r)
x3 = np.zeros(K2)
x4 = np.zeros(K2b)
x5 = np.zeros(M2b)

In [51]:
err = np.zeros(M2b)

In [52]:
dedw1 = np.zeros(K1b)
dedw2 = np.zeros(K1r)
dedw3 = np.zeros(K2)
dedw4 = np.zeros(K2b)
dedw5 = np.zeros(M2b)

In [53]:
numVal = 0
for n in range(Nvalidate):
    x0 = np.copy(xValidate[:, n])
    x1 = np.dot(w0, x0)
    x2 = np.add(x1, w1)
    x3 = np.maximum(0, x2)
    x4 = np.dot(w3, x3)
    x5 = np.add(x4, w4)
    
    k_ = np.argmax(x5)
    if yValidate[n] != k_:
        numVal = numVal + 1

numVal = (100.0*numVal)/Nvalidate
print("Initial Accur. (Validation):\t{0:6.2f}".format(100.0 - numVal))

Initial Accur. (Validation):	 11.55


In [54]:
eIndex = 0
eNum   = 0
eSum   = 0.0

for epoch in range(epochs):
    for n in range(Ntrain):
        x0 = np.copy(xTrain[:, n])
        x1 = np.dot(w0, x0)
        x2 = np.add(x1, w1)
        x3 = np.maximum(0, x2)
        x4 = np.dot(w3, x3)
        x5 = np.add(x4, w4)
        
        stark = yTrain[n]
        px           = np.subtract(x5, np.max(x5))    
        px           = np.exp(px)                     
        px           = np.true_divide(px, np.sum(px)) 
        e            = -np.log(px[stark])             
        dedw5        = np.copy(px)                   
        dedw5[stark] = dedw5[stark] - 1.0

        k_ = np.argmax(x5)
        if stark != k_:
            eNum = eNum + 1
            
        eSum = eSum + e
        eIndex = eIndex + 1
        
        if eIndex == EF:
            eNum = (100.0*eNum)/EF
            eSum = (1.0*eSum)/EF
            print("Accur. & Err (training):\t{0:6.2f}\t{1:6.2f}".format(100.0 - eNum, eSum))
            eIndex = 0
            eNum   = 0
            eSum   = 0.0
        
        dedw4 = np.copy(dedw5)
        dedw3 = np.dot(w3.T, dedw4)
        dedw2 = np.multiply(np.minimum(1.0, np.ceil(np.maximum(0.0, x2))), dedw3)
        dedw1 = np.copy(dedw2)
        
        dedw0 = np.outer(dedw1, x0)
        dedw1 = np.copy(dedw2)
        dedw3 = np.outer(dedw4, x3)
        dedw4 = np.copy(dedw5)

        w0 = np.subtract(w0, np.multiply(LR, dedw0))
        w1 = np.subtract(w1, np.multiply(LR, dedw1))
        w3 = np.subtract(w3, np.multiply(LR, dedw3))
        w4 = np.subtract(w4, np.multiply(LR, dedw4))
        
    eNumVal = 0
    for n in range(Nvalidate):
        
        x0 = np.copy(xValidate[:, n])
        x1 = np.dot(w0, x0)
        x2 = np.add(x1, w1)
        x3 = np.maximum(0, x2)
        x4 = np.dot(w3, x3)
        x5 = np.add(x4, w4)

        k_ = np.argmax(x5)
        if yValidate[n] != k_:
            eNumVal = eNumVal + 1
    eNumVal = (100.0*eNumVal)/Nvalidate
    print("Current Accur. (validation):\t{0:6.2f}".format(100.0 - eNumVal))
    LR = LS*LR

Accur. & Err (training):	 48.56	  2.23
Accur. & Err (training):	 58.59	  1.33
Accur. & Err (training):	 59.53	  1.40
Accur. & Err (training):	 60.03	  1.36
Accur. & Err (training):	 63.22	  1.22
Current Accur. (validation):	 69.48
Accur. & Err (training):	 76.57	  0.92
Accur. & Err (training):	 78.02	  0.82
Accur. & Err (training):	 81.81	  0.78
Accur. & Err (training):	 81.94	  0.75
Accur. & Err (training):	 82.95	  0.69
Current Accur. (validation):	 86.91
Accur. & Err (training):	 86.61	  0.52
Accur. & Err (training):	 87.27	  0.47
Accur. & Err (training):	 88.02	  0.46
Accur. & Err (training):	 87.90	  0.47
Accur. & Err (training):	 88.37	  0.42
Current Accur. (validation):	 90.34
Accur. & Err (training):	 90.01	  0.38
Accur. & Err (training):	 89.83	  0.37
Accur. & Err (training):	 90.52	  0.35
Accur. & Err (training):	 90.06	  0.36
Accur. & Err (training):	 90.48	  0.33
Current Accur. (validation):	 91.50
Accur. & Err (training):	 91.11	  0.32
Accur. & Err (training):	 90.96	  0.3

In [55]:
numTest = 0
for n in range(Ntest):
    x0 = np.copy(xTest[:, n])
    x1 = np.dot(w0, x0)
    x2 = np.add(x1, w1)
    x3 = np.maximum(0, x2)
    x4 = np.dot(w3, x3)
    x5 = np.add(x4, w4)
    
    k_ = np.argmax(x5)
    if yTest[n] != k_:
        numTest = numTest + 1
numTest = (100.0*numTest)/Ntest
print("Accuracy (testing):\t{0:6.2f}".format(100.0 - numTest))

Accuracy (testing):	 91.21
